In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import math
from tqdm import tqdm
pd.options.display.float_format = '{:5f}'.format
pd.set_option('display.max_columns', 40)

from plotnine import ggplot, aes, geom_line, facet_wrap, facet_grid, theme, element_text, geom_bar, stat_count, geom_point, \
geom_boxplot, geom_jitter, geom_violin, ggtitle, element_blank, scale_y_continuous

In [21]:
# Load a file
df_overlap = pd.read_pickle('/aihub/data/raw/df_overlap_with_people.pkl')

# Extract only location history between 9am and 10pm

In [ ]:
# Date the positive/negative cases to when they co-exist
df_overlap['pcr_result'] = df_overlap['pcr_result'].astype(int)
df_overlap['pcr_date'] = pd.to_datetime(df_overlap['pcr_date'])
df_overlap2 = df_overlap[df_overlap['pcr_date'].isin(pd.date_range('2022-02-15', '2022-05-31'))].reset_index(drop = True)
# Extract only between 9am and 10pm
df_overlap3 = df_overlap2[df_overlap2['hour'].isin(range(9, 23))].reset_index(drop = True)
# add a duration column per location record
df_overlap3['end_time'] = pd.to_datetime(df_overlap3['end_time'])
df_overlap3['start_time'] = pd.to_datetime(df_overlap3['start_time'])
df_overlap3['stay_time'] = df_overlap3['end_time'] - df_overlap3['start_time']

In [16]:
# set type
df_overlap['pcr_result'] = df_overlap['pcr_result'].astype(int)
df_overlap['pcr_date'] = pd.to_datetime(df_overlap['pcr_date'])
# Extract only between 9am and 10pm
df_overlap3 = df_overlap2.copy()
display(df_overlap3.head(3))
print(df_overlap3['id'].nunique())
# add a duration column per location record
df_overlap3['end_time'] = pd.to_datetime(df_overlap3['end_time'])
df_overlap3['start_time'] = pd.to_datetime(df_overlap3['start_time'])
df_overlap3['stay_time'] = df_overlap3['end_time'] - df_overlap3['start_time']
df_overlap3.head()

,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender,last_vac_date,stay_time,month,hour,weekday_cri,loc_id,loc_type,overlap_cnt,overlaped_seconds,overlaped_people
0,46104,2022-03-22 12:37:58,2022-03-22 12:38:34,33.116939,126.268908,2,2022-03-24,2 days,NaT,NaT,2022-01-27,NaT,56.000000,1996-09-03,서울특별시 종로구 명륜1가,1.000000,NaT,0 days 00:00:36,3,12,4,33.116938_126.268908,outside,0,[],[]
1,34827,2022-02-24 10:47:23,2022-02-24 10:47:50,33.200194,126.270711,2,2022-03-02,6 days,2021-08-30,2021-10-05,NaT,NaT,148.000000,1995-11-08,서울특별시 구로구 가리봉동,1.000000,2021-10-05,0 days 00:00:27,2,10,4,33.200194_126.270711,outside,0,[],[]
2,60173,2022-04-10 18:57:10,2022-04-10 18:57:18,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:08,4,18,5,33.200194_126.270711,outside,0,[],[]


804


,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender,last_vac_date,stay_time,month,hour,weekday_cri,loc_id,loc_type,overlap_cnt,overlaped_seconds,overlaped_people
0,46104,2022-03-22 12:37:58,2022-03-22 12:38:34,33.116939,126.268908,2,2022-03-24,2 days,NaT,NaT,2022-01-27,NaT,56.000000,1996-09-03,서울특별시 종로구 명륜1가,1.000000,NaT,0 days 00:00:36,3,12,4,33.116938_126.268908,outside,0,[],[]
1,34827,2022-02-24 10:47:23,2022-02-24 10:47:50,33.200194,126.270711,2,2022-03-02,6 days,2021-08-30,2021-10-05,NaT,NaT,148.000000,1995-11-08,서울특별시 구로구 가리봉동,1.000000,2021-10-05,0 days 00:00:27,2,10,4,33.200194_126.270711,outside,0,[],[]
2,60173,2022-04-10 18:57:10,2022-04-10 18:57:18,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:08,4,18,5,33.200194_126.270711,outside,0,[],[]
3,60173,2022-04-10 18:55:05,2022-04-10 18:55:06,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:01,4,18,5,33.200194_126.270711,outside,0,[],[]
4,60173,2022-04-10 17:30:47,2022-04-10 17:30:48,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:01,4,17,5,33.200194_126.270711,outside,0,[],[]


In [17]:
min_criteria = 1/6 # 10 seconds in 1 min

df_overlap3['overlap_total_cnt'] = 0
df_overlap3['overlap_total_seconds'] = 0
df_overlap3['overlap_max_seconds'] = 0
df_overlap3['overlap_avg_seconds'] = 0
df_overlap3['overlap_min_seconds'] = 0
df_overlap3.reset_index(drop=True, inplace=True)

seconds_criteria = 60 * min_criteria
for i, row in df_overlap3.iterrows():
    tmp = [x for x in row['overlaped_seconds'] if x >= seconds_criteria]
    if tmp:
        df_overlap3['overlap_total_cnt'].iloc[i] = len(tmp)
        df_overlap3['overlap_total_seconds'].iloc[i] = sum(tmp)
        df_overlap3['overlap_max_seconds'].iloc[i] = max(tmp)
        df_overlap3['overlap_avg_seconds'].iloc[i] = sum(tmp)/len(tmp)
        df_overlap3['overlap_min_seconds'].iloc[i] = min(tmp)
df_overlap3.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,start_time,end_time,latitude,longitude,pcr_result,pcr_date,date_diff,dose_1_date,dose_2_date,dose_3_date,dose_4_date,vac_date_diff,birthday,address,gender,last_vac_date,stay_time,month,hour,weekday_cri,loc_id,loc_type,overlap_cnt,overlaped_seconds,overlaped_people,overlap_total_cnt,overlap_total_seconds,overlap_max_seconds,overlap_avg_seconds,overlap_min_seconds
0,46104,2022-03-22 12:37:58,2022-03-22 12:38:34,33.116939,126.268908,2,2022-03-24,2 days,NaT,NaT,2022-01-27,NaT,56.000000,1996-09-03,서울특별시 종로구 명륜1가,1.000000,NaT,0 days 00:00:36,3,12,4,33.116938_126.268908,outside,0,[],[],0,0,0,0.000000,0
1,34827,2022-02-24 10:47:23,2022-02-24 10:47:50,33.200194,126.270711,2,2022-03-02,6 days,2021-08-30,2021-10-05,NaT,NaT,148.000000,1995-11-08,서울특별시 구로구 가리봉동,1.000000,2021-10-05,0 days 00:00:27,2,10,4,33.200194_126.270711,outside,0,[],[],0,0,0,0.000000,0
2,60173,2022-04-10 18:57:10,2022-04-10 18:57:18,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:08,4,18,5,33.200194_126.270711,outside,0,[],[],0,0,0,0.000000,0
3,60173,2022-04-10 18:55:05,2022-04-10 18:55:06,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:01,4,18,5,33.200194_126.270711,outside,0,[],[],0,0,0,0.000000,0
4,60173,2022-04-10 17:30:47,2022-04-10 17:30:48,33.200194,126.270711,2,2022-04-12,2 days,2021-08-18,2021-09-17,2022-02-07,NaT,64.000000,1995-04-13,서울특별시 중구 황학동,1.000000,2022-02-07,0 days 00:00:01,4,17,5,33.200194_126.270711,outside,0,[],[],0,0,0,0.000000,0


In [ ]:
# Function for creating a variable for each person using location overlap information
def overlap_seconds_summary(df):
    result = df.agg({
        'loc_id': 'nunique', 'stay_time': 'sum', 
        'overlap_total_cnt': 'sum', 'overlap_total_seconds': 'sum', 'overlap_max_seconds': 'max',
        'overlap_avg_seconds' : 'mean'
    })
    result['overlap_min_seconds'] = 0
    result['overlap_location'] = 0
    result['overlap_people'] = 0
    
    # Counting overlapped places (unique)
    tmp1 = df[df['overlap_cnt'] > 0].copy()
    if tmp1.shape[0] > 0:
        result['overlap_location'] = tmp1['loc_id'].nunique()
        
    tmp2 = df[df['overlap_min_seconds'] > 0].copy()
    if tmp2.shape[0] > 0:
        result['overlap_min_seconds'] = tmp2['overlap_min_seconds'].min()
    
    df.reset_index(drop=True, inplace=True)
    tmp_overlap_people = []
    for _, row in df.iterrows():
        if row['overlaped_people']:
            tmp_overlap_people.extend(row['overlaped_people'])
    result['overlap_people'] = len(set(tmp_overlap_people))
        
    return result

In [18]:
df_summarised = df_overlap3.groupby(['pcr_result', 'id', 'loc_type'], as_index=False).apply(lambda x: overlap_seconds_summary(x))
print(df_summarised.info())
#df_summarised[df_summarised['overlap_total_cnt'] > 0]
df_summarised['id'] = df_summarised['id'].map(lambda x: str(int(x)))
df_summarised['stay_time'] = df_summarised['stay_time'].map(lambda x: x.seconds)
df_summarised = df_summarised.pivot(index=['pcr_result', 'id'], columns='loc_type', \
                                    values=['overlap_total_cnt', \
                                            'overlap_total_seconds', 'overlap_max_seconds', \
                                            'overlap_avg_seconds', 'overlap_min_seconds', 'overlap_location', 'overlap_people'])
df_summarised.reset_index(inplace=True)
print(df_summarised.info())
#df_summarised.drop(['overlap_cnt', 'stay_time'], axis=1, inplace=True)
df_summarised.columns = df_summarised.columns.to_flat_index()
df_summarised.columns = ['pcr_result', 'id', \
                        'overlap_cnt_outside', 'overlap_cnt_resi', \
                         'overlap_seconds_outside', 'overlap_seconds_resi', \
                         'overlap_max_seconds_outside', 'overlap_max_seconds_resi', 'overlap_avg_seconds_outside', \
                         'overlap_avg_seconds_resi', 'overlap_min_seconds_outside', 'overlap_min_seconds_resi', \
                        'overlap_location_outside', 'overlap_location_resi', 'overlap_people_outside', 'overlap_people_resi']
df_summarised.drop(['overlap_location_resi'], axis=1, inplace=True)
df_summarised

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1608 entries, 0 to 1607
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   pcr_result             1608 non-null   int64          
 1   id                     1608 non-null   object         
 2   loc_type               1608 non-null   object         
 3   loc_id                 1608 non-null   int64          
 4   stay_time              1608 non-null   timedelta64[ns]
 5   overlap_total_cnt      1608 non-null   int64          
 6   overlap_total_seconds  1608 non-null   int64          
 7   overlap_max_seconds    1608 non-null   int64          
 8   overlap_avg_seconds    1608 non-null   float64        
 9   overlap_min_seconds    1608 non-null   int64          
 10  overlap_location       1608 non-null   int64          
 11  overlap_people         1608 non-null   int64          
dtypes: float64(1), int64(8), object(2), timedelta64[

,pcr_result,id,overlap_cnt_outside,overlap_cnt_resi,overlap_seconds_outside,overlap_seconds_resi,overlap_max_seconds_outside,overlap_max_seconds_resi,overlap_avg_seconds_outside,overlap_avg_seconds_resi,overlap_min_seconds_outside,overlap_min_seconds_resi,overlap_location_outside,overlap_people_outside,overlap_people_resi
0,1,21084,0.000000,1.000000,0.000000,34.000000,0.000000,34.000000,0.000000,0.018299,0.000000,34.000000,0.000000,0.000000,1.000000
1,1,22193,619.000000,445.000000,14731.000000,10496.000000,230.000000,430.000000,3.443218,4.393753,10.000000,10.000000,27.000000,5.000000,3.000000
2,1,22911,211.000000,658.000000,6298.000000,17948.000000,228.000000,430.000000,1.665580,4.104650,10.000000,10.000000,28.000000,3.000000,2.000000
3,1,22991,8.000000,0.000000,109.000000,0.000000,20.000000,0.000000,0.059956,0.000000,10.000000,0.000000,2.000000,1.000000,0.000000
4,1,23008,1.000000,0.000000,47.000000,0.000000,47.000000,0.000000,0.015030,0.000000,47.000000,0.000000,1.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,2,63393,2.000000,0.000000,33.000000,0.000000,17.000000,0.000000,0.006587,0.000000,16.000000,0.000000,1.000000,1.000000,0.000000
800,2,63962,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
801,2,64153,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
802,2,64189,0.000000,1.000000,0.000000,35.000000,0.000000,35.000000,0.000000,0.028926,0.000000,35.000000,0.000000,0.000000,1.000000


In [19]:
df_summarised.to_pickle('/aihub/data/raw/final_df/df_overlap_10s_daytime.pkl')